# Import and sandbox

In [1]:
import numpy as np
import pandas as pd
import torch 
from tqdm import tqdm
import matplotlib.pyplot as plt


# check n_wells 

## Imports

In [ ]:

df_phenom = pd.read_parquet('/projects/synsight/data/jump_embeddings/wells_embeddings/openphenom/metadata_openphenom.parquet')

In [3]:
df_phenom = pd.read_parquet('/home/maxime/data/jump_embeddings/metadata_dinov2_g.parquet')


In [4]:
df_phenom.drop(columns=['Metadata_Well', 'Metadata_JCP2022', 'Metadata_InChI'], inplace=True)

In [5]:
df_phenom = df_phenom.drop_duplicates()

In [ ]:
min_samples_per_source = df_phenom.groupby('Metadata_Source').size().min()

balanced_sample = df_phenom.groupby('Metadata_Source').apply(
    lambda group: group.groupby('Metadata_Batch').apply(
        lambda batch: batch.sample(n=min(len(batch), min_samples_per_source // len(group['Metadata_Batch'].unique())))
    ).reset_index(drop=True)
).reset_index(drop=True)



In [ ]:
balanced_sample['Metadata_Source'].value_counts()


In [ ]:
import pandas as pd

# Calculer le nombre maximum de plaques échantillonnées uniformément
min_samples_per_source = df_phenom.groupby('Metadata_Source').size().min()

# Échantillonner un nombre fixe de plaques par source
balanced_sample = df_phenom.groupby('Metadata_Source').apply(
    lambda group: group.sample(n=min_samples_per_source, random_state=42)  # Fixer un seed pour reproductibilité
).reset_index(drop=True)

balanced_sample['Metadata_Source'].value_counts()

In [ ]:
import pandas as pd

# Calculate the number of plates per source to maintain balance
n_sources = df_phenom['Metadata_Source'].nunique()

# Ensure a minimum number of plates per source if possible
def balanced_sample_with_total_limit(df, total_limit=48):
    n_sources = df['Metadata_Source'].nunique()
    plates_per_source = total_limit // n_sources

    # First, sample equally across sources
    sampled = (
        df.groupby('Metadata_Source', group_keys=False).apply(
            lambda source_group: source_group.sample(
                n=min(len(source_group), plates_per_source), random_state=42)
        )
    )

    # If we have fewer than the total_limit, fill the remaining slots
    if len(sampled) < total_limit:
        remaining_slots = total_limit - len(sampled)
        remaining_sample = df.loc[~df.index.isin(sampled.index)].sample(n=remaining_slots, random_state=42)
        sampled = pd.concat([sampled, remaining_sample])

    return sampled

# Apply the function
balanced_sample = balanced_sample_with_total_limit(df_phenom, total_limit=48)


In [ ]:
balanced_sample['Metadata_Source'].value_counts()

In [ ]:
balanced_sample.info()

# Export and save

In [11]:
balanced_sample.to_json("balanced_plates_388_plate.json", orient="records")


# Distribution 

In [4]:
elected_plates = pd.read_json(
            "/home/maxime/synrepos/phenoseeker/scripts/balanced_plates.json"
        )

In [ ]:
elected_plates

In [ ]:
elected_plates['Metadata_Source'].value_counts()